# Visualizing Graph of Computations with Tensorboard

A very usefull part of the Tensorflow framework is the Tensorboard web application that is a visualization helper. Amongst others, it helps the researcher to inspect the computational graph and monitor the training process through graphs.



(*Objective: Use Tensorboard in Colab, familiarize computational graph/scopes and implement a basic neural network. Time: 12 mins*) 


## Installing in Google Collab

In Google collab we can install libraries and Python packages in the virtual environment. 

We are going to need the [ngrok](https://ngrok.com/) library to expose our virtual localhost to a public URL

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip

--2019-08-25 22:43:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.54.84.112, 3.212.234.252, 34.224.236.142, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.54.84.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  18.5MB/s    in 0.7s    

2019-08-25 22:43:45 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Below there is an example of Python package installation. The Colab runtime has most common packages preinstalled.

In [0]:
! pip install xlwt

## Multilayer Perceptron (MLP) Neural Network 
<img src="https://miro.medium.com/max/958/1*QVIyc5HnGDWTNX3m-nIm9w.png" width="300" border="1"/>

The source code creates the computational graph for a simple Multi-Layer Perceptron (MLP) neural network. It has an input vector with 3072 dimensions, 1024 neurons in the hidden layer and 10 neurons in the output layer. The graph is exported to a Tensorboard log file.



In [0]:
# NEURAL NETWORK - Spaghetti Code

import tensorflow as tf
import math

TENSORBOARD_FOLDER = "/tmp/tboard"

oGraph = tf.Graph()
with oGraph.as_default():
  
  # ........ Input Layer .......
  tInput = tf.placeholder(tf.uint8, shape=(100,3072))
  tInputNormalized = tf.cast(tInput, tf.float32) / tf.constant(255.0, tf.float32)
  tX = tInputNormalized
  
  # ........ Hidden Layer (128 Neurons) .......
  tWeight1      = tf.Variable(tf.random.truncated_normal([3072,1024], stddev=math.sqrt(2/(3072+1024))), tf.float32)
  tBias1        = tf.Variable(tf.zeros(1024), tf.float32)
  tSynapticSum1 = tf.matmul(tX, tWeight1) + tBias1
  # Hidden layer has the sigmoid activation function
  tActivation1  = tf.nn.sigmoid(tSynapticSum1)
  
  # ........ Output Layer .......
  tWeight2      = tf.Variable(tf.random.truncated_normal([1024,10], stddev=math.sqrt(2/(1024+10))), tf.float32)
  tBias2        = tf.Variable(tf.zeros(10), tf.float32)
  tSynapticSum2 = tf.matmul(tActivation1, tWeight2) + tBias2
  # Output layer has the sigmoid activation function
  tActivation2  = tf.nn.sigmoid(tSynapticSum2)
  
  
  with tf.Session() as oSession:
    assert oSession.graph == oGraph, "The current session is using the default graph"
    oSession.run(tf.initializers.global_variables())
  
  
    oWriter = tf.summary.FileWriter(TENSORBOARD_FOLDER, graph=oSession.graph, flush_secs=20)
    oWriter.flush()
    
    print("Graph exported to %s" % TENSORBOARD_FOLDER)

W0825 22:53:08.998400 140369036949376 meta_graph.py:449] Issue encountered when serializing variables.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool
W0825 22:53:09.000277 140369036949376 meta_graph.py:449] Issue encountered when serializing trainable_variables.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool


Graph exported to /tmp/tboard


## Tensorboard

We start tensorboard and tunneling using ngrok. The output of tensorboard becomes available at the returned URL.



In [0]:
# kill all running ngrok instances
!pkill -f ngrok
!pkill -f tensorboard

# Execute tensorboard
LOG_DIR = '/tmp/tboard/'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(TENSORBOARD_FOLDER))

# execute ngrok
get_ipython().system_raw('./ngrok http 6006 &')

# Do the tunneling
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://19ee03f0.ngrok.io


## Contextual organization with variable scopes

Inspecting the graph of the first example, we notice that the model parameters (variables) are automatically named and that the tree lacks a contextual organization. The code is rewritten bellow, using variable scopes and custom variable names. 

In [0]:
# NEURAL NETWORK - Spaghetti Code with Scopes

import tensorflow as tf
import math

TENSORBOARD_FOLDER = "/tmp/tboard2"

oGraph = tf.Graph()
with oGraph.as_default():
  with tf.variable_scope("NeuralNet"):
    with tf.variable_scope("Input"):
      tInput = tf.placeholder(tf.uint8, shape=(100,3072))
      tInputNormalized = tf.cast(tInput, tf.float32) / tf.constant(255.0, tf.float32)
      tX = tInputNormalized
      
    with tf.variable_scope("FC1"):
      tWeight1      = tf.get_variable("w", shape=[3072,1024], initializer=tf.initializers.truncated_normal(mean=0.0, stddev=math.sqrt(2/(3072+1024))), dtype=tf.float32)
      tBias1        = tf.get_variable("b", shape=[1024], initializer=tf.initializers.constant(0.0, dtype=tf.float32))
      tSynapticSum1 = tf.matmul(tX, tWeight1) + tBias1
      # Hidden layer has the sigmoid activation function
      tActivation1  = tf.nn.sigmoid(tSynapticSum1)
  

    with tf.variable_scope("FC2"):
      # ........ Output Layer .......
      tWeight2      = tf.get_variable("w", shape=[1024,10], initializer=tf.initializers.truncated_normal(mean=0.0, stddev=math.sqrt(2/(1024+10))), dtype=tf.float32)
      tBias2        = tf.get_variable("b", shape=[10], initializer=tf.initializers.constant(0.0), dtype=tf.float32)
      tSynapticSum2 = tf.matmul(tActivation1, tWeight2) + tBias2
      # Output layer has the sigmoid activation function
      tActivation2  = tf.nn.sigmoid(tSynapticSum2)
  
  
  with tf.Session() as oSession:
    assert oSession.graph == oGraph, "The current session is using the default graph"
    oSession.run(tf.initializers.global_variables())
  
  
    oWriter = tf.summary.FileWriter(TENSORBOARD_FOLDER, graph=oSession.graph, flush_secs=20)
    oWriter.flush()
    
    print("Graph exported to %s" % TENSORBOARD_FOLDER)

W0825 22:57:51.151877 140369036949376 deprecation.py:506] From <ipython-input-14-62fb8479aa7a>:17: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Graph exported to /tmp/tboard2


Restarting tensorboard will display a contextually organized graph. Notice how the model parameters are named.

In [0]:
# kill all running ngrok instances
!pkill -f ngrok
!pkill -f tensorboard

# Execute tensorboard
LOG_DIR = '/tmp/tboard2/'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(TENSORBOARD_FOLDER))

# execute ngrok
get_ipython().system_raw('./ngrok http 6006 &')

# Do the tunneling
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ee923f10.ngrok.io
